In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from scipy import stats

In [ ]:
df = pd.read_csv('data/togo.csv')  # Example

In [ ]:
# Summary and missing values
df.describe()
df.isna().sum()

In [ ]:
# Handle missing values
df.fillna(df.median(), inplace=True)

In [ ]:
# Outlier detection
z_scores = np.abs(stats.zscore(df.select_dtypes('number')))
df = df[(z_scores < 3).all(axis=1)]

In [ ]:
# Save cleaned data
df.to_csv('data/togo_clean.csv', index=False)

In [ ]:
# Plot examples
sns.lineplot(x='Timestamp', y='GHI', data=df)
plt.title('Togo - Solar Radiation Over Time')
plt.show()